<a href="https://colab.research.google.com/github/blancavazquez/CursoDatosMasivosII/blob/2023-I/notebooks/6b_spectral_clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Agrupamiento espectral
En esta libreta programaremos el método de agrupamiento espectral y lo aplicaremos al descubrimiento de comunidades en redes sociales.

In [ ]:
import networkx as nx
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import random
from sklearn.cluster import KMeans, SpectralClustering

np.random.seed(2022)
n_componentes = 4

## Grafo simple
Empezamos creando un grafo simple, del cual analizaremos el [espectro](https://en.wikipedia.org/wiki/Spectrum_of_a_matrix) de su matriz laplaciana.

In [ ]:
G_simple = nx.Graph()
G_simple.add_nodes_from(range(10))
acom1 = [(i,j)for i in range(4) for j in range(4) if i != j]
acom2 = [(i,j)for i in range(4,8) for j in range(4,8) if i != j]
acom3 = [(i,j)for i in range(8,12) for j in range(8,12) if i != j]
G_simple.add_edges_from(acom1)
G_simple.add_edges_from(acom2)
G_simple.add_edges_from(acom3)
G_simple.add_edge(3,4)
G_simple.add_edge(2,5)
G_simple.add_edge(6,8)
G_simple.add_edge(7,9)
G_simple.add_edge(10,12)
G_simple.add_edge(11,12)
G_simple.add_edge(12,13)

Visualizamos el grafo creado.

In [ ]:
simple_pos = nx.spring_layout(G_simple)
nx.draw(G_simple, pos=simple_pos)

Obtenemos y desplegamos la matriz de adyacencia del grafo.

In [ ]:
amat_simple = nx.linalg.adjacency_matrix(G_simple)
print(amat_simple.toarray())

Hacemos lo propio con la matriz de grado.

In [ ]:
dmat_simple = np.diag(nx.linalg.laplacian_matrix(G_simple).diagonal())
print(dmat_simple)
print(dmat_simple.diagonal())

Finalmente obtenemos y desplegamos la matriz laplaciana.

In [ ]:
lmat_simple = nx.linalg.laplacian_matrix(G_simple)
print(lmat_simple.toarray())

Calculamos los vectores y valores propios de la matriz laplaciana.

In [ ]:
lmat_simple_sp= sp.sparse.csgraph.laplacian(amat_simple)
print(lmat_simple_sp.toarray())

In [ ]:
eval_simple, evec_simple = sp.sparse.linalg.eigsh(lmat_simple.astype(np.float64),
                                                  sigma=1.0, 
                                                  which="LM",
                                                  tol=0.0,
                                                  v0= np.random.uniform(-1, 1, size=lmat_simple.shape[0]))

In [ ]:
print(evec_simple[:, 1])
print(np.sort(evec_simple[:, 1]))

Verificamos que las restricciones se están cumpliendo.

In [ ]:
print(np.sum(evec_simple**2, axis=0))
print(np.sum(evec_simple, axis=0))

Ordenamos los valores propios de menor a mayor y mantenemos el mismo orden para los vectores propios correspondientes.

In [ ]:
orden_idx = np.argsort(eval_simple)
eval_simple = eval_simple[orden_idx]
evec_simple = evec_simple[:, orden_idx]

Examinamos los valores propios ordenados.

In [ ]:
print(eval_simple)

Desplegamos el primer valor propio y su correspondiente vector propio. Nota que el valor propio más pequeño $\approx 0$ y que todos los elementos de su vector propio correspondiente son iguales.

In [ ]:
print(eval_simple[0])
print(evec_simple[:, 0])

Visualizamos los elementos de los vectores propios con valor propio más pequeño.

In [ ]:
fig, axs = plt.subplots(1,4, figsize=(15, 4), sharey=True)

for i in range(4):
  axs[i].plot(np.sort(evec_simple[:, i]), 'bo')
  axs[i].set_ylim([-1,1])
  axs[i].set_xlabel('Orden en el vector propio')
  axs[i].set_title('$\\lambda_' + str(i + 1) + '$')

axs[0].set_ylabel('Valor')
fig.tight_layout()
plt.show()

Empleamos KMeans para agrupar los nodos usando los eigenvalores asociados.

In [ ]:
km_simple = KMeans(n_clusters=3, n_init=100)
km_simple = km_simple.fit(evec_simple[:, 1:])
km_simple.labels_

In [ ]:
print(G_simple.nodes)

In [ ]:
mapa_color = {0:'blue', 1:'red', 2:'green'}
color_vertices = [mapa_color[km_simple.labels_[n]] for n in G_simple]
nx.draw(G_simple, pos=simple_pos, node_color=color_vertices)

## Club de karate de Zachary
Hacemos el mismo análisis del grafo anterior usando el grafo del [Club de karate de Zachary](https://en.wikipedia.org/wiki/Zachary%27s_karate_club), el cual está disponible en la biblioteca NetworkX. 

Primero, cargamos y visualizamos el grafo.


In [ ]:
G_karate = nx.karate_club_graph()    
karate_pos = nx.spring_layout(G_karate)
nx.draw(G_karate)

Obtenemos su matriz de adyacencia y laplaciana.

In [ ]:
lmat_karate = nx.linalg.laplacian_matrix(G_karate)
amat_karate = nx.linalg.adjacency_matrix(G_karate)
lmat_karate = lmat_karate.toarray()

In [ ]:
amat_karate.toarray()

Calculamos los vectores y valores propios de la matriz lapaciana.

In [ ]:
eval_karate, evec_karate = np.linalg.eigh(lmat_karate.astype(np.float64))

Examinamos sus propiedades.

In [ ]:
print(np.sum(evec_karate**2, axis=0))
print(np.sum(evec_karate, axis=0))

Ordenamos los vectores propios en orden ascendente al valor propio asociado.

In [ ]:
orden_idx = np.argsort(eval_karate)
eval_karate = eval_karate[orden_idx]
evec_karate = evec_karate[:, orden_idx]

Desplegamos los valores propios ordenados.

In [ ]:
print(eval_karate)

Examinamos el vector propio asociado al valor propio más pequeño.

In [ ]:
print(eval_karate[0])
print(evec_karate[:, 0])

Graficamos los elementos de los vectores propios asociados a los cuatro valores propios más pequeños.

In [ ]:
fig, axs = plt.subplots(1,4, figsize=(15, 4), sharey=True)

for i in range(4):
  axs[i].plot(np.sort(evec_karate[:, i]), 'bo')
  axs[i].set_ylim([-1,1])
  axs[i].set_xlabel('Orden en el vector propio')
  axs[i].set_title('$\\lambda_' + str(i + 1) + '$')

axs[0].set_ylabel('Valor')
fig.tight_layout()
plt.show()

Agrupamos los elementos del vector propio con valor propio más pequeño.

In [ ]:
km_karate = KMeans(n_clusters=3, n_init=100)
km_karate = km_karate.fit(evec_karate[:, [1]])
km_karate.labels_

Visualizamos el agrupamiento de los vértices.

In [ ]:
color_vertices = [mapa_color[km_karate.labels_[n]] for n in G_karate]
nx.draw(G_karate, pos=karate_pos, node_color=color_vertices)

## Detección de comunidades en _Facebook's Social circles_
Aplicamos agrupamiento espectral a los [círculos sociales de Facebook de SNAP](https://snap.stanford.edu/data/ego-Facebook.html).

Primero descargamos y desempaquetamos los datos.

In [ ]:
!wget https://snap.stanford.edu/data/facebook_combined.txt.gz
!gunzip facebook_combined.txt.gz

Cargamos el grafo y lo visualizamos.

In [ ]:
G_large = nx.read_edgelist('facebook_combined.txt')
large_pos = nx.spring_layout(G_large)
nx.draw(G_large, pos=large_pos)

Obtenemos su matriz laplaciana y le aplicamos agrupamiento espectral para obtener 10 comunidades

In [ ]:
amat_large = nx.linalg.adjacency_matrix(G_large)
sc_large = SpectralClustering(n_clusters=10, 
                              affinity='precomputed', 
                              n_init=100)
sc_large = sc_large.fit(amat_large)

Visualizamos las comunidades

In [ ]:
nx.draw(G_large, pos=large_pos, node_color=sc_large.labels_, cmap=plt.cm.jet)